# Structured Output with Pydantic V2

Simple example of getting structured data from agents using Pydantic V2.

## Key Concept
Instead of free-form text → Get validated, structured data objects

In [8]:
from pydantic import BaseModel, Field
from langchain_ollama import ChatOllama
from langchain.agents import create_agent
# import tools

## Define Data Structure

In [9]:
class ContactInfo(BaseModel):
    name: str
    email: str
    phone: str


class ListofContactInfo(BaseModel):
    contacts: list[ContactInfo] = Field(..., description="List of contact information")

## Test Simple Structured Output

In [10]:
model = ChatOllama(model="qwen3")

agent = create_agent(
    model,
    tools=[], # if case if you don't want to use any tools, just pass an empty list
    response_format=ContactInfo
)

result = agent.invoke({
    "messages":"Extract contact info from: John Doe, john@example.com, (555) 123-4567, Jon Do, john@example.com1, (555) 123"
})

result

{'messages': [HumanMessage(content='Extract contact info from: John Doe, john@example.com, (555) 123-4567, Jon Do, john@example.com1, (555) 123', additional_kwargs={}, response_metadata={}, id='ad877bed-4da7-4d2b-904d-8009902a2662'),
  AIMessage(content='<think>\nOkay, let\'s see. The user wants me to extract contact info from the given text. The input is: John Doe, john@example.com, (555) 123-4567, Jon Do, john@example.com1, (555) 123.\n\nFirst, I need to parse this data. The entries are separated by commas, but there might be some inconsistencies. Let me list them out:\n\n1. John Doe\n2. john@example.com\n3. (555) 123-4567\n4. Jon Do\n5. john@example.com1\n6. (555) 123\n\nWait, the fourth entry is "Jon Do" which looks like a name, but the third entry is a phone number. The fifth entry is an email with "com1" which might be a typo. The sixth is a phone number but incomplete, missing the last few digits.\n\nSo, the user provided two names, two emails, and two phone numbers. But some en

## Test Complex Structured Output

In [11]:
model = ChatOllama(model="qwen3")

agent = create_agent(
    model,
    tools=[], # if case if you don't want to use any tools, just pass an empty list
    response_format=ListofContactInfo
)

result = agent.invoke({
    "messages":"Extract contact info from: John Doe, john@example.com, (555) 123-4567, Jon Do, john@example.com1, (555) 123"
})
result

{'messages': [HumanMessage(content='Extract contact info from: John Doe, john@example.com, (555) 123-4567, Jon Do, john@example.com1, (555) 123', additional_kwargs={}, response_metadata={}, id='3b45cf0f-1a98-4d15-9c04-9994db6efa8d'),
  AIMessage(content='<think>\nOkay, let\'s see. The user wants me to extract contact info from the given text. The input is: John Doe, john@example.com, (555) 123-4567, Jon Do, john@example.com1, (555) 123.\n\nFirst, I need to parse this data. It looks like the information is separated by commas. Let me split it into individual parts. So, the first part is "John Doe" which is likely the name. Then the email is "john@example.com". Next is the phone number "(555) 123-4567". Then there\'s "Jon Do" which might be another name, possibly a typo or a different entry. The next part is "john@example.com1" which seems like an email but has a number at the end, maybe an error. The last part is "(555) 123" which is a phone number but missing the last four digits.\n\nN

In [17]:
print(result['structured_response'].contacts[0])
print(result['structured_response'].contacts[1])

name='John Doe' email='john@example.com' phone='(555) 123-4567'
name='Jon Do' email='john@example.com1' phone='(555) 123'
